<a href="https://colab.research.google.com/github/tumurzakov/latentflow/blob/main/demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install diffusers transformers einops git+https://github.com/openai/CLIP.git torchvision compel mediapy OmegaConf accelerate xformers decord[torch]
!git clone https://github.com/tumurzakov/latentflow.git
!git clone https://github.com/tumurzakov/AnimateDiff.git

!wget -c "https://huggingface.co/guoyww/animatediff/resolve/main/mm_sd_v15_v2.ckpt?download=true" -O "/content/mm_sd_v15_v2.ckpt"

!git lfs install
!git clone https://huggingface.co/runwayml/stable-diffusion-v1-5 --branch fp16 --depth 1


  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-i1igx_mh
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-i1igx_mh
  Resolved https://github.com/openai/CLIP.git to commit a1d071733d7111c9c014f024669f959182114e33
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 MB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.4/53.4 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 9.6 MB/s eta 0:00:00
  Preparing metadata 

Cloning into 'latentflow'...
remote: Enumerating objects: 67, done.
remote: Counting objects: 100% (67/67), done.
remote: Compressing objects: 100% (56/56), done.
remote: Total 67 (delta 12), reused 65 (delta 10), pack-reused 0
Receiving objects: 100% (67/67), 316.00 KiB | 10.53 MiB/s, done.
Resolving deltas: 100% (12/12), done.
Cloning into 'AnimateDiff'...
remote: Enumerating objects: 1820, done.
remote: Counting objects: 100% (313/313), done.
remote: Compressing objects: 100% (116/116), done.
remote: Total 1820 (delta 220), reused 274 (delta 190), pack-reused 1507
Receiving objects: 100% (1820/1820), 39.28 MiB | 27.44 MiB/s, done.
Resolving deltas: 100% (928/928), done.
--2024-01-09 22:21:25--  https://huggingface.co/guoyww/animatediff/resolve/main/mm_sd_v15_v2.ckpt?download=true
Resolving huggingface.co (huggingface.co)... 18.239.50.103, 18.239.50.16, 18.239.50.49, ...
Connecting to huggingface.co (huggingface.co)|18.239.50.103|:443... connected.
HTTP request sent, awaiting respons

In [2]:
%%writefile /content/inference.yaml

unet_additional_kwargs:
  unet_use_cross_frame_attention: false
  unet_use_temporal_attention: false
  use_motion_module: true
  motion_module_resolutions:
  - 1
  - 2
  - 4
  - 8
  motion_module_mid_block: false
  motion_module_decoder_only: false
  motion_module_type: Vanilla
  motion_module_kwargs:
    num_attention_heads: 8
    num_transformer_block: 1
    attention_block_types:
    - Temporal_Self
    - Temporal_Self
    temporal_position_encoding: true
    temporal_position_encoding_max_len: 32
    temporal_attention_dim_div: 1

noise_scheduler_kwargs:
  DDIMScheduler:
    num_train_timesteps: 1000
    beta_start: 0.00085
    beta_end: 0.012
    beta_schedule: "linear"
    steps_offset: 1
    clip_sample: false

  EulerAncestralDiscreteScheduler:
    num_train_timesteps: 1000
    beta_start: 0.00085
    beta_end: 0.012
    beta_schedule: "linear"

  KDPM2AncestralDiscreteScheduler:
    num_train_timesteps: 1000
    beta_start: 0.00085
    beta_end: 0.012
    beta_schedule: "linear"


Writing /content/inference.yaml


In [3]:
import sys
if 'AnimateDiff' not in sys.path:
  sys.path.insert(0, '/content/latentflow/src')
  sys.path.insert(0, '/content/AnimateDiff')

In [4]:
from omegaconf import OmegaConf
from diffusers import AutoencoderKL, EulerAncestralDiscreteScheduler
from transformers import CLIPTextModel, CLIPTokenizer
from animatediff.models.unet import UNet3DConditionModel
from animatediff.pipelines.pipeline_animation import AnimationPipeline

def load_pipeline(
    inference_config_path,
    pretrained_model_path,
    motion_module_path,
):
  inference_config = OmegaConf.load(inference_config_path)

  tokenizer = CLIPTokenizer.from_pretrained(pretrained_model_path, subfolder="tokenizer")

  text_encoder = CLIPTextModel.from_pretrained(pretrained_model_path, subfolder="text_encoder").to('cuda')

  vae = AutoencoderKL.from_pretrained(pretrained_model_path, subfolder="vae").to('cuda')

  unet = UNet3DConditionModel.from_pretrained_2d(
          pretrained_model_path,
          subfolder="unet",
          unet_additional_kwargs=OmegaConf.to_container(inference_config.unet_additional_kwargs))

  scheduler = EulerAncestralDiscreteScheduler(
          **OmegaConf.to_container(
              inference_config.noise_scheduler_kwargs['EulerAncestralDiscreteScheduler']
              )
          )

  pipeline = AnimationPipeline(
          unet=unet,
          text_encoder=text_encoder,
          tokenizer=tokenizer,
          vae=vae,
          scheduler=scheduler,
          )

  motion_module_state_dict = torch.load(motion_module_path, map_location="cpu")
  missing, unexpected = pipeline.unet.load_state_dict(motion_module_state_dict, strict=False)
  assert len(unexpected) == 0

  return pipeline


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [5]:
import torch

pipe = load_pipeline(
    '/content/inference.yaml',
    '/content/stable-diffusion-v1-5',
    '/content/mm_sd_v15_v2.ckpt')

loaded temporal unet's pretrained weights from /content/stable-diffusion-v1-5/unet ...
### missing keys: 560; 
### unexpected keys: 0;
### Temporal Module Parameters: 417.1376 M


/content/AnimateDiff/animatediff/pipelines/pipeline_animation.py:245: FutureWarning: The configuration file of this scheduler: EulerAncestralDiscreteScheduler {
  "_class_name": "EulerAncestralDiscreteScheduler",
  "_diffusers_version": "0.25.0",
  "beta_end": 0.012,
  "beta_schedule": "linear",
  "beta_start": 0.00085,
  "num_train_timesteps": 1000,
  "prediction_type": "epsilon",
  "rescale_betas_zero_snr": false,
  "steps_offset": 0,
  "timestep_spacing": "linspace",
  "trained_betas": null
}
 is outdated. `steps_offset` should be set to 1 instead of 0. Please make sure to update the config accordingly as leaving `steps_offset` might led to incorrect results in future versions. If you have downloaded this checkpoint from the Hugging Face Hub, it would be very nice if you could open a Pull request for the `scheduler/scheduler_config.json` file
  deprecate("steps_offset!=1", "1.0.0", deprecation_message, standard_warn=False)


In [6]:
%cd /content/latentflow

from latentflow.state import State
from latentflow.video import Video
from latentflow.latent import Latent
from latentflow.tile import Tile
from latentflow.prompt import Prompt
from latentflow.vae_video_encode import VaeVideoEncode
from latentflow.vae_latent_decode import VaeLatentDecode
from latentflow.diffuse import Diffuse
from latentflow.unet import Unet
from latentflow.prompt_embeddings import PromptEmbeddings
from latentflow.a1111_prompt_encode import A1111PromptEncode
from latentflow.compel_prompt_encode import CompelPromptEncode
from latentflow.video_load import VideoLoad
from latentflow.video_show import VideoShow
from latentflow.latent_show import LatentShow
from latentflow.noise import Noise
from latentflow.add_noise import AddNoise
from latentflow.schedule import Schedule
from latentflow.debug import Debug
from latentflow.invert import Invert
from latentflow.bypass import Bypass

import importlib
import latentflow.video_load
importlib.reload(latentflow.video_load)
from latentflow.video_load import VideoLoad

unet = Unet(
    unet=pipe.unet.to('cuda'),
    scheduler=pipe.scheduler,
    guidance_scale=10,
)

state = State()
video = \
    (Video(video=torch.randn((16,288,512,3))) \
        - Debug("Video to video") \
        - VideoLoad('/content/latentflow/src/tests_func/bunny.mp4', device='cuda', video_length=16) \
        - VideoShow(fps=16) \
        - VaeVideoEncode(vae=pipe.vae.to('cuda')) \
        - Invert(tokenizer=pipe.tokenizer, text_encoder=pipe.text_encoder, unet=pipe.unet, scheduler=pipe.scheduler) \
        - LatentShow(fps=16, vae=pipe.vae.to('cuda')) \
        < state("latent") \
        ) >> \
    (Latent(latent=torch.randn((1,4,16,36,64)), device='cuda') \
        | Debug("Text to video") \
        | Noise(scheduler=pipe.scheduler) \
        | LatentShow(fps=16, vae=pipe.vae.to('cuda')) \
        > state("latent") \
        ) >> \
    (Prompt("a cat walking down city street") \
        | CompelPromptEncode(tokenizer=pipe.tokenizer, text_encoder=pipe.text_encoder.to('cuda')) \
        > state("embeddings") \
        ) >> \
    (state \
        | Schedule(scheduler=pipe.scheduler, num_inference_steps=10) \
        > state('timesteps') \
        ) >> \
    (state['latent'] \
        - Debug("Adding noise")
        - AddNoise(scheduler=pipe.scheduler) \
        < state('latent') \
        ) >> \
    (state
        | Debug("Diffusion loop") \
        | Diffuse(callback=lambda timestep, state: state | unet)
        | VaeLatentDecode(vae=pipe.vae.to('cuda'))
    ) \
    | VideoShow(fps=16)

/content/latentflow


This browser does not support the video tag.


This browser does not support the video tag.
